In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import  OneHotEncoder
from sklearn.model_selection import train_test_split
import pandas as pd

In [2]:
df = pd.read_csv("./mma.csv")

In [3]:
selected_cols = [
    "WeightClass",
    "Gender",
    "Age",
    "Reach",
    "StrikingSkill",
    "GrapplingSkill",
    "SubmissionSkill",
    "Experience",
    "LastFightResult",
    "FightingStyle",
]

In [5]:
encoder = OneHotEncoder(sparse_output=False)
encoder.fit(df[["WeightClass", "Gender", "LastFightResult", "FightingStyle"]])


def onehot_encode(df, encoder):
    return pd.DataFrame(encoder.transform(df), columns=encoder.get_feature_names_out())

In [6]:
encoded_df = onehot_encode(
    df=df[["WeightClass", "Gender", "LastFightResult", "FightingStyle"]],
    encoder=encoder,
)
encoded_df

,WeightClass_Bantamweight,WeightClass_Featherweight,WeightClass_Flyweight,WeightClass_Heavyweight,WeightClass_Light Heavyweight,WeightClass_Lightweight,WeightClass_Middleweight,WeightClass_Strawweight,WeightClass_Welterweight,Gender_female,Gender_male,LastFightResult_Loss,LastFightResult_Win,FightingStyle_ All-Rounder,FightingStyle_All-Rounder,FightingStyle_Grappler,FightingStyle_Striker,FightingStyle_Striker ```plaintext,FightingStyle_Wrestler
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
66,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
67,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
68,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [ ]:
X_train_df = pd.concat(
    [
        df.drop(columns=["Winner"])[
            [
                # "WeightClass",
                # "Gender",
                "Age",
                "Reach",
                "StrikingSkill",
                "GrapplingSkill",
                "SubmissionSkill",
                "Experience",
                # "LastFightResult",
                # "FightingStyle",
            ]
        ],
        encoded_df,
    ],
    axis=1,
)

X_train, X_test, Y_train, Y_test = train_test_split(
    X_train_df,
    df[["Winner"]],
    test_size=0.2,
)

X_train.head()

,Age,Reach,StrikingSkill,GrapplingSkill,SubmissionSkill,Experience,WeightClass_Bantamweight,WeightClass_Featherweight,WeightClass_Flyweight,WeightClass_Heavyweight,...,Gender_female,Gender_male,LastFightResult_Loss,LastFightResult_Win,FightingStyle_ All-Rounder,FightingStyle_All-Rounder,FightingStyle_Grappler,FightingStyle_Striker,FightingStyle_Striker ```plaintext,FightingStyle_Wrestler
8,35,80,75,70,65,30,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
10,32,80,90,70,75,20,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
63,28,67,80,75,70,10,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
49,37,74,65,80,55,50,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
43,30,74,85,75,65,20,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [9]:
model = DecisionTreeClassifier()
model.fit(X_train, Y_train)

print(f"Accuracy: {model.score(X_test, Y_test)}")

Accuracy: 1.0


In [10]:
in_weight_class = input("WeightClass:")
in_gender = input("Gender (male/female):")
in_age = input("Age:")
in_reach = input("Reach:")
in_striking_skill = input("StrikingSkill:")
in_graping_skill = input("GrampingSkill:")
in_submission_skill = input("SubmissionSkill:")
in_experience = input("Experience:")
in_last_fight_result = input("Last Fight Result (Win/Loss):")
in_fighting_style = input("Fighting Style:")

test_in = pd.DataFrame(
    # [["Welterweight", "male", 30, 74, 85, 75, 65, 20, "Loss", "All-Rounder"]],
    # [["Bantamweight", "male", 30, 70, 80, 75, 65, 15, "Win", "All-Rounder"]],
    [
        [
            in_weight_class,
            in_gender,
            in_age,
            in_reach,
            in_striking_skill,
            in_graping_skill,
            in_submission_skill,
            in_experience,
            in_last_fight_result,
            in_fighting_style,
        ]
    ],
    columns=selected_cols,
)

encoded_test_in = onehot_encode(
    test_in[["WeightClass", "Gender", "LastFightResult", "FightingStyle"]],
    encoder=encoder,
)
input = pd.concat(
    [
        test_in[
            [
                "Age",
                "Reach",
                "StrikingSkill",
                "GrapplingSkill",
                "SubmissionSkill",
                "Experience",
            ]
        ],
        encoded_test_in,
    ],
    axis=1,
)

if model.predict(input)[0] == "YES":
    print("Win")
else:
    print("Loss")

Loss


In [11]:
if model.predict(input)[0] == "YES":
    print("Win")
else:
    print("Loss")

Loss
